## آموزش مدل CBoW

این نوت‌بوک بخشی از [برنامه درسی هوش مصنوعی برای مبتدیان](http://aka.ms/ai-beginners) است.

در این مثال، ما به آموزش مدل زبانی CBoW می‌پردازیم تا فضای تعبیه Word2Vec خودمان را ایجاد کنیم. برای این کار از مجموعه داده AG News به عنوان منبع متن استفاده خواهیم کرد.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ابتدا بیایید مجموعه داده خود را بارگذاری کنیم و توکنایزر و واژگان را تعریف کنیم. ما مقدار `vocab_size` را به ۵۰۰۰ تنظیم می‌کنیم تا محاسبات کمی محدود شود.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## مدل CBoW

مدل CBoW یاد می‌گیرد که یک کلمه را بر اساس $2N$ کلمات همسایه پیش‌بینی کند. برای مثال، وقتی $N=1$ باشد، از جمله *I like to train networks* جفت‌های زیر را خواهیم داشت: (like,I)، (I, like)، (to, like)، (like,to)، (train,to)، (to, train)، (networks, train)، (train,networks). در اینجا، کلمه اول کلمه همسایه‌ای است که به عنوان ورودی استفاده می‌شود و کلمه دوم همان کلمه‌ای است که پیش‌بینی می‌کنیم.

برای ساخت یک شبکه که کلمه بعدی را پیش‌بینی کند، باید کلمه همسایه را به عنوان ورودی ارائه دهیم و شماره کلمه را به عنوان خروجی دریافت کنیم. معماری شبکه CBoW به صورت زیر است:

* کلمه ورودی از لایه تعبیه عبور می‌کند. این لایه تعبیه همان تعبیه Word2Vec ما خواهد بود، بنابراین آن را به صورت جداگانه به عنوان متغیر `embedder` تعریف می‌کنیم. در این مثال از اندازه تعبیه = 30 استفاده می‌کنیم، هرچند ممکن است بخواهید با ابعاد بالاتر آزمایش کنید (Word2Vec واقعی دارای 300 است).
* بردار تعبیه سپس به یک لایه خطی منتقل می‌شود که کلمه خروجی را پیش‌بینی می‌کند. بنابراین این لایه دارای تعداد نورون‌های `vocab_size` است.

برای خروجی، اگر از `CrossEntropyLoss` به عنوان تابع خطا استفاده کنیم، باید فقط شماره کلمات را به عنوان نتایج مورد انتظار ارائه دهیم، بدون استفاده از کدگذاری یک‌داغ.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## آماده‌سازی داده‌های آموزشی

حالا بیایید تابع اصلی را برنامه‌نویسی کنیم که زوج‌های کلمات CBoW را از متن محاسبه می‌کند. این تابع به ما امکان می‌دهد اندازه پنجره را مشخص کنیم و مجموعه‌ای از زوج‌ها - کلمه ورودی و خروجی - را برگرداند. توجه داشته باشید که این تابع می‌تواند هم روی کلمات و هم روی بردارها/تنسورها استفاده شود - که به ما اجازه می‌دهد متن را رمزگذاری کنیم، قبل از اینکه آن را به تابع `to_cbow` ارسال کنیم.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


بیایید مجموعه داده آموزشی را آماده کنیم. ما تمام اخبار را مرور می‌کنیم، `to_cbow` را فراخوانی می‌کنیم تا لیست جفت‌های کلمات را دریافت کنیم و آن جفت‌ها را به `X` و `Y` اضافه می‌کنیم. به خاطر صرفه‌جویی در زمان، فقط ۱۰ هزار خبر اول را در نظر می‌گیریم - شما می‌توانید به راحتی این محدودیت را حذف کنید اگر زمان بیشتری برای انتظار دارید و می‌خواهید تعبیه‌های بهتری دریافت کنید :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

ما همچنین آن داده‌ها را به یک مجموعه داده تبدیل می‌کنیم و یک بارگذار داده ایجاد می‌کنیم:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

ما همچنین آن داده‌ها را به یک مجموعه داده تبدیل می‌کنیم و یک بارگذار داده ایجاد می‌کنیم:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

حالا بیایید آموزش واقعی را شروع کنیم. ما از بهینه‌ساز `SGD` با نرخ یادگیری نسبتاً بالا استفاده خواهیم کرد. همچنین می‌توانید با سایر بهینه‌سازها مانند `Adam` آزمایش کنید. ما ابتدا برای ۱۰ دوره آموزش خواهیم داد - و اگر می‌خواهید خطای کمتری داشته باشید، می‌توانید این سلول را دوباره اجرا کنید.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## امتحان Word2Vec

برای استفاده از Word2Vec، بیایید بردارهای مربوط به تمام کلمات موجود در واژگان خود را استخراج کنیم:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

بیایید ببینیم، به عنوان مثال، چگونه کلمه **پاریس** به یک بردار رمزگذاری می‌شود:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


جالب است که از Word2Vec برای یافتن مترادف‌ها استفاده کنیم. تابع زیر `n` کلمه نزدیک به ورودی داده شده را بازمی‌گرداند. برای یافتن آن‌ها، نُرم $|w_i - v|$ را محاسبه می‌کنیم، که در آن $v$ بردار مربوط به کلمه ورودی ما است و $w_i$ کدگذاری کلمه $i$-ام در واژگان است. سپس آرایه را مرتب کرده و شاخص‌های مربوطه را با استفاده از `argsort` بازمی‌گردانیم و اولین `n` عنصر لیست را که موقعیت کلمات نزدیک در واژگان را کدگذاری می‌کنند، انتخاب می‌کنیم.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## نکات مهم

با استفاده از تکنیک‌های هوشمندانه مانند CBoW، می‌توانیم مدل Word2Vec را آموزش دهیم. همچنین می‌توانید مدل skip-gram را امتحان کنید که برای پیش‌بینی کلمات همسایه با توجه به کلمه مرکزی آموزش داده می‌شود و ببینید عملکرد آن چقدر خوب است.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
